In [1]:
from gitsource import GithubRepositoryDataReader, chunk_documents
from minsearch import AppendableIndex


reader = GithubRepositoryDataReader(
    repo_owner="evidentlyai",
    repo_name="docs",
    allowed_extensions={"md", "mdx"},
)
files = reader.read()

parsed_docs = [doc.parse() for doc in files]
chunked_docs = chunk_documents(parsed_docs, size=3000, step=1500)

index = AppendableIndex(
    text_fields=["title", "description", "content"],
    keyword_fields=["filename"]
)
index.fit(chunked_docs)

In [7]:
def search(query: str) -> list[dict]:
    """
    Search the documentation database for relevant results based on a query string.

    Args:
        query (str): The search query to look up in the index.

    Returns:
        list[dict]: A list of search result objects returned by the index.
    """
    results = index.search(
        query=query,
        num_results=5
    )
    return results

def add_entry(filename, title, description, content):
    """
    Add a new documentation entry to the index.

    Args:
        filename (str): The source filename associated with the entry.
        title (str): The title of the documentation entry.
        description (str): A short description summarizing the entry.
        content (str): The full content of the documentation entry.

    Returns:
        str: A status message indicating the result of the operation ("OK").
    """
    entry = {
        'start': 0,
        'content': content,
        'title': title,
        'description': description,
        'filename': filename,
    }
    index.append(entry)
    return "OK"


In [12]:
instructions = """
You're a documentation assistant. 

Answer the user question using the documentation knowledge base

Make 3 iterations:

1) in the first iteration, perform one search
2) in the second interation, analyze the results from the previous search
   and perform 2 more searches
3) synthesise the results into the output

IMPORTANT: at each step, give an explanation of why you want to perform 
search for this particular search query. It should be 2-3 sentences explaining
the logic of your decision.

Use only facts from the knowledge base when answering.
If you cannot find the answer, inform the user.

Our knowledge base is entirely about Evidently, so you don't need to 
include the word 'evidently' in search results
"""

In [13]:
from toyaikit.tools import Tools

agent_tools = Tools()
agent_tools.add_tool(search)
agent_tools.add_tool(add_entry)

agent_tools.get_tools()

[{'type': 'function',
  'name': 'search',
  'description': 'Search the documentation database for relevant results based on a query string.\n\nArgs:\n    query (str): The search query to look up in the index.\n\nReturns:\n    list[dict]: A list of search result objects returned by the index.',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'query parameter'}},
   'required': ['query'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'add_entry',
  'description': 'Add a new documentation entry to the index.\n\nArgs:\n    filename (str): The source filename associated with the entry.\n    title (str): The title of the documentation entry.\n    description (str): A short description summarizing the entry.\n    content (str): The full content of the documentation entry.\n\nReturns:\n    str: A status message indicating the result of the operation ("OK").',
  'parameters': {'type': 'object',
   'properties': {'filenam

In [14]:
from openai import OpenAI
from toyaikit.llm import OpenAIClient

from toyaikit.chat.runners import OpenAIResponsesRunner
from toyaikit.chat.interface import IPythonChatInterface
from toyaikit.chat.runners import DisplayingRunnerCallback

In [15]:
llm_client = OpenAIClient(
    model="gpt-4o-mini",
    client=OpenAI()
)

chat_interface = IPythonChatInterface()
runner_callback = DisplayingRunnerCallback(chat_interface=chat_interface) 

In [16]:
agent = OpenAIResponsesRunner(
    tools=agent_tools,
    developer_prompt=instructions,
    chat_interface=chat_interface,
    llm_client=llm_client
)

In [18]:
result = agent.loop(
    'how can I create evidently dahsbord?',
    callback=runner_callback
)

-> Response received


-> Response received


-> Response received


Parameter,Type,Required,Default,Description
title,str,Yes,None,The title of the panel.
size,str,Yes,"""full""","Panel size: ""full"" (100% width) or ""half"" (50%)."
values,list,Yes,—,List of PanelMetric objects to display.
tab,str,Yes,None,Dashboard tab name; defaults to the first tab or creates a new tab if necessary.
plot_params,dict,No,{},"Panel visualization settings like ""plot_type"" (e.g., ""text"", ""line"", ""counter"")."


In [22]:
class SearchIndexTools:

    def __init__(self, index): 
        self.index = index
    
    def search(self, query: str) -> list[dict]:
        """
        Search the documentation database for relevant results based on a query string.
    
        Args:
            query (str): The search query to look up in the index.
    
        Returns:
            list[dict]: A list of search result objects returned by the index.
        """
        results = self.index.search(
            query=query,
            num_results=5
        )
        return results
    
    def add_entry(self, filename, title, description, content):
        """
        Add a new documentation entry to the index.
    
        Args:
            filename (str): The source filename associated with the entry.
            title (str): The title of the documentation entry.
            description (str): A short description summarizing the entry.
            content (str): The full content of the documentation entry.
    
        Returns:
            str: A status message indicating the result of the operation ("OK").
        """
        entry = {
            'start': 0,
            'content': content,
            'title': title,
            'description': description,
            'filename': filename,
        }
        self.index.append(entry)
        return "OK"


In [26]:
search_tools_object = SearchIndexTools(index)
results = search_tools_object.search('dashboard')

In [27]:
agent_tools = Tools()
agent_tools.add_tool(search_tools_object.search)
agent_tools.add_tool(search_tools_object.add_entry)

In [31]:
agent_tools = Tools()
agent_tools.add_tools(search_tools_object)

In [32]:
agent = OpenAIResponsesRunner(
    tools=agent_tools,
    developer_prompt=instructions,
    chat_interface=chat_interface,
    llm_client=llm_client
)

result = agent.loop(
    'how can I create evidently dahsbord?',
    callback=runner_callback
)

-> Response received


-> Response received


-> Response received


In [33]:
import search_tools

In [35]:
search_tools_object = search_tools.SearchIndexTools(index)
result = search_tools_object.search('dashboard')